In [1]:
from image_series import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
# from skimage import feature, measure, restoration
from defects import *
from natsort import natsorted
import glob


In [2]:

path = r"C:\Users\user\Desktop\work\test\raw"
IS = image_series(path)

In [3]:
IS.run_all()

100%|██████████| 220/220 [02:35<00:00,  1.42it/s]


In [6]:
    def velocity_averaging(self, window=None):
        """
        takes all the images in the velocity and defect folders, crops a window around each
        defect and tilts the velocity field around the defect.
        save
        """
        window = self.velocity_around_defect_window if window is None else window
        half_window = np.floor(self.velocity_around_defect_window/2)
        final_size = int(np.floor(half_window / np.sqrt(2)))

        mean_arr_plus = np.zeros(shape=(final_size,final_size,2))
        mean_arr_minus = np.zeros_like(mean_arr_plus)
        total_number_of_minus,total_number_of_plus = 0,0
        minus_defect_num = []
        plus_defect_num = []
        for img_num,img in enumerate(self.images):
            #there is 1 less flow image thus we skip the last image
            if img_num +1 >= len(self.images):
                break

            # get the dataframe and the array
            min_path = self.defects_csv_path + img.name + '_MinusHalf.csv'
            plus_path = self.defects_csv_path + img.name + '_PlusHalf.csv'
            flow_path = self.velocity_path + 'velocity_from_' + img.name + '.npy'
            minus_df = pd.read_csv(min_path, header=0, index_col=0)
            plus_df = pd.read_csv(plus_path, header=0, index_col=0)
            flow_arr = np.load(flow_path)
            #minus
            number_of_defects_minus, mean_arr_minus = img.crop_and_tilt( defects_df=minus_df,
                                velocity_array= flow_arr,
                                half_window=half_window,
                                save= self.save_velocity,
                                path=self.velocity_around_minus_path)
            #plus
            number_of_defects_plus, mean_arr_plus = img.crop_and_tilt( defects_df=plus_df,
                                velocity_array=flow_arr,
                                half_window=half_window,
                                save=self.save_velocity,
                                path=self.velocity_around_plus_path)

            total_number_of_plus += number_of_defects_plus
            total_number_of_minus += number_of_defects_minus
            plus_defect_num.append(number_of_defects_plus)
            minus_defect_num.append(number_of_defects_minus)

            #save
            if self.save_velocity:
                np.save(self.velocity_around_minus_path + '_mean_arr_from_' + img.name + '.npy',mean_arr_minus)
                np.save(self.velocity_around_plus_path + '_mean_arr_from_' + img.name + '.npy',mean_arr_plus)

        #calculate average over the mean arrays
        final_plus = np.zeros_like(mean_arr_plus)
        final_minus = np.zeros_like(mean_arr_minus)

        #add plus
        array_list = glob.glob(self.velocity_around_plus_path + r"\*npy")
        array_list = natsorted(array_list, key=lambda y: y.lower())

        for idx,arr_path in enumerate(array_list):

            #there is 1 less flow image thus we skip the last image
            if img_num +1 >= len(self.images):
                break

            arr = np.load(arr_path)
            arr = arr * plus_defect_num[idx] / total_number_of_plus
            final_plus += arr

        #add minus
        array_list = glob.glob(self.velocity_around_minus_path + r"\*npy")
        array_list = natsorted(array_list, key=lambda y: y.lower())

        for idx,arr_path in enumerate(array_list):

            #there is 1 less flow image thus we skip the last image
            if img_num +1 >= len(self.images):
                break
            arr = np.load(arr_path)
            arr = arr * minus_defect_num[idx] / total_number_of_minus
            final_minus += arr

        # save final
        if self.save_velocity:
            np.save(self.velocity_around_minus_path + 'final_average_minus' + '.npy',final_minus)
            np.save(self.velocity_around_plus_path + 'final_average_plus' + '.npy',final_plus)
